In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.layers import (Dense, BatchNormalization)
from tensorflow import keras

file_path = "./Dataset"

# 경로 내 모든 파일을 딕셔너리 형태로 읽음
def get_csv(file_path):
  tmp = dict()
  for file in os.listdir(file_path): # os.listdir(file_path) : 파일 list를 가지고 옴
    file_name = file[:-4] # .csv 확장자를 제외한 파일 이름 추출
    tmp[file_name] = pd.read_csv(file_path + "/" + file) # 파일 경로에 해당하는 csv파일을 가지고 옴

  return tmp

# 데이터 프레임을 기준에 맞게 하나로 합침
def merge(dict_data):
  test_Data = pd.merge(dict_data["gender_submission"], dict_data["test"], how="outer", on="PassengerId") # 두 데이터 프레임을 id를 기준으로 해서 하나로 합침
  mergeData = pd.concat([dict_data["train"], test_Data]) # 두 데이터 프레임을 행 기준으로 합침
  mergeData.reset_index(drop=True, inplace=True) # drop = 인덱스로 설정한 열을 데이터 프레임에서 삭제, inplace = 원본 객체 변경 여부

  return mergeData

from copy import copy
# list에 해당하는 열 제거
def remove_col(remove_list):
  result = copy(merge_dataSet) # 원본 복사

  result.set_index("PassengerId", inplace=True) # 승객 번호를 index로 지정

  for col in remove_list:
    del(result[col])

  return result

# 폴더 내 csv 파일 읽어옴
dataSet = get_csv(file_path)
# csv 파일들을 하나로 합침
merge_dataSet = merge(dataSet)
# 학습에 필요없는 값들 제거
remove_list = ["Name", "Ticket", "Cabin", "Embarked"]
df_data = remove_col(remove_list)
# 결측 값 제거
df_data.dropna(inplace=True)
# 학습을 위해 문자를 정수로 변환
df_data["Sex"] = np.where(df_data["Sex"].to_numpy() == "male", 0, 1)

# Min-Max Scaling, 최소 값은 0, 최대 값을 1로 모든 데이터를 0 ~ 1 범위 안에 들어가도록 조절함
age_min, age_max = np.min(df_data["Age"]), np.max(df_data["Age"])
fare_min, fare_max = np.min(df_data["Fare"]), np.max(df_data["Fare"])
df_data["Age"] = (df_data["Age"] - age_min) / (age_max - age_min)
df_data["Fare"] = (df_data["Fare"] - age_min) / (age_max - age_min)

# test데이터를 200개 그 외의 값들은 모두 train으로 사용
y_test, y_train = df_data["Survived"][:200].to_numpy(), df_data["Survived"][200:].to_numpy()
del(df_data["Survived"])
x_test, x_train = df_data[:200].values, df_data[200:].values
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
print("x_train: ", len(x_train), "  x_test: ", len(x_test))
# 학습률 설정
learning_rate = 0.01

# 순차적 구조 모델 생성
model = keras.Sequential()
# 배치 정규화, 각 층의 입력을 정규화하여 학습 속도와 안정성을 증가시킴
model.add(BatchNormalization())
# 모델에 레이어 층 생성
model.add(Dense(32, input_dim=7, activation = "relu"))
model.add(Dense(16, activation = "relu"))
model.add(Dense(4, activation = "relu"))
model.add(Dense(1, activation = "sigmoid"))

# 모델 학습 전 최적화 함수, 손실 함수, 평가 지표 등 설정
optimizer = keras.optimizers.Adam(learning_rate)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["binary_accuracy"])

# 학습 진행
model.fit(x_train, y_train, batch_size=64, epochs=100)

# 평가
prediction = model.predict(x_test).reshape(x_test.shape[0])
prediction = tf.cast(prediction > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, y_test), dtype=np.float32))

# 정확도 출력
print("accuracy: ", accuracy.numpy())


x_train:  845   x_test:  200
Epoch 1/100
14/14 [==============================] - 1s 3ms/step - loss: 0.5298 - binary_accuracy: 0.7917
Epoch 2/100
14/14 [==============================] - 0s 3ms/step - loss: 0.3743 - binary_accuracy: 0.8615
Epoch 3/100
14/14 [==============================] - 0s 3ms/step - loss: 0.3721 - binary_accuracy: 0.8615
Epoch 4/100
14/14 [==============================] - 0s 3ms/step - loss: 0.3884 - binary_accuracy: 0.8402
Epoch 5/100
14/14 [==============================] - 0s 3ms/step - loss: 0.3618 - binary_accuracy: 0.8556
Epoch 6/100
14/14 [==============================] - 0s 4ms/step - loss: 0.3366 - binary_accuracy: 0.8734
Epoch 7/100
14/14 [==============================] - 0s 3ms/step - loss: 0.3405 - binary_accuracy: 0.8746
Epoch 8/100
14/14 [==============================] - 0s 3ms/step - loss: 0.3417 - binary_accuracy: 0.8710
Epoch 9/100
14/14 [==============================] - 0s 3ms/step - loss: 0.3338 - binary_accuracy: 0.8781
Epoch 10/100
14/1